# Variational AutoEncoder (MLP) Pytorch
Train a variational autoencoder with MNIST dataset

#### References:
* http://kvfrans.com/variational-autoencoders-explained/
* https://github.com/pytorch/examples/blob/master/mnist/main.py
* https://github.com/pytorch/examples/blob/master/vae/main.py
* http://wiseodd.github.io/techblog/2017/01/24/vae-pytorch/
* https://github.com/ritheshkumar95/pytorch-vqvae
* https://pytorch.org/docs/stable/distributions.html
* https://ermongroup.github.io/cs228-notes/extras/vae/

In [1]:
import os
import torch
import torch.utils.data
from torch import nn, optim
from torch.nn import functional as F
from torch.distributions.normal import Normal
from torchvision import datasets, transforms
from torchvision.utils import save_image
#from torch.utils.tensorboard import SummaryWriter
batch_size = 128
ZDIMS = 10
num_hidden = 800
epochs = 10
num_classes = 10
print("PyTorch Version: ",torch.__version__)

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print('Device:', device)
num_gpu = torch.cuda.device_count()
print('Number of available GPUs:', num_gpu)

log_interval = 10

PyTorch Version:  1.0.0
Device: cpu
Number of available GPUs: 0


#### Download Data

In [2]:
# Download or load downloaded MNIST dataset
# shuffle data at every epoch
train_loader = torch.utils.data.DataLoader(datasets.MNIST('data', 
    train=True, download=True,transform=transforms.ToTensor()),
    batch_size=batch_size, shuffle=True)

# Same for test data
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('data', train=False, transform=transforms.ToTensor()),
    batch_size=batch_size, shuffle=False)

#### Define Model

In [3]:
class VAE(nn.Module):
    def __init__(self):
        super(VAE, self).__init__()
        # Encoder
        self.fc1 = nn.Linear((28*28), num_hidden)
        self.fc20 = nn.Linear(num_hidden, num_hidden)
        self.fc21 = nn.Linear(num_hidden, ZDIMS)
        self.fc22 = nn.Linear(num_hidden, ZDIMS)
        
        # Decoder
        self.fc3 = nn.Linear(ZDIMS, num_hidden)
        self.fc31 = nn.Linear(num_hidden, num_hidden)
        self.fc4 = nn.Linear(num_hidden, (28*28))


    def encode(self, x):
        h1 = F.relu(self.fc1(x))
        h1_2 = F.relu(self.fc20(h1))
        return self.fc21(h1_2), self.fc22(h1_2)
    
    def decode(self, z):
        h3 = F.relu(self.fc3(z))
        h31 = F.relu(self.fc31(h3))
        return torch.sigmoid(self.fc4(h31))

    def reparameterize(self, mu, logvar):
        std = torch.exp(0.5*logvar)
        # Sample from normal 0 mean unit variance distribution
        eps = torch.randn_like(std)
        # Reparametrization trick
        guessed_z = mu + (eps*std)
        return guessed_z

    def forward(self, x):
        mu, logvar = self.encode(x.view(-1, (28*28)))
        # Differentiable (Reparametrization Trick)
        z = self.reparameterize(mu, logvar)
        # Non-differentiable
        #z = torch.normal(mu, logvar)
        return self.decode(z), mu, logvar

In [4]:
model = VAE().to(device)
optimizer = optim.Adam(model.parameters(), lr=1e-3)

#### Define Loss Function

In [5]:
# Reconstruction + KL divergence losses summed over all elements and batch
def loss_function(recon_x, x, mu, logvar):
    BCE = F.binary_cross_entropy(recon_x, x.view(-1, 784), reduction='sum')
    KLD = -0.5 * torch.sum(1 + logvar - mu.pow(2) - logvar.exp())
    return BCE + KLD

#### Define Train/Test Functions

In [6]:
def train(epoch):
    model.train()
    train_loss = 0
    for batch_idx, (data, _) in enumerate(train_loader):
        data = data.to(device)
        optimizer.zero_grad()
        recon_batch, mu, logvar = model(data)
        loss = loss_function(recon_batch, data, mu, logvar)
        loss.backward()
        train_loss += loss.item()
        optimizer.step()
        if batch_idx % log_interval == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader),
                loss.item() / len(data)))

    print('====> Epoch: {} Average loss: {:.4f}'.format(
          epoch, train_loss / len(train_loader.dataset)))


def test(epoch):
    model.eval()
    test_loss = 0
    with torch.no_grad():
        for i, (data, _) in enumerate(test_loader):
            data = data.to(device)
            recon_batch, mu, logvar = model(data)
            test_loss += loss_function(recon_batch, data, mu, logvar).item()
            if i == 0:
                n = min(data.size(0), 8)
                comparison = torch.cat([data[:n],
                                      recon_batch.view(batch_size, 1, 28, 28)[:n]])
                save_image(comparison.cpu(),
                         'results/reconstruction_' + str(epoch) + '.png', nrow=n)

    test_loss /= len(test_loader.dataset)
    print('====> Test set loss: {:.4f}'.format(test_loss))

#### Train/Test

In [7]:
# 206 loss
for epoch in range(1, epochs + 1):
        train(epoch)
        test(epoch)
        with torch.no_grad():
            z = torch.randn(64, ZDIMS).to(device)
            sample = model.decode(z).cpu()
            save_image(sample.view(64, 1, 28, 28),
                       'results/sample_' + str(epoch) + '.png')

Train Epoch: 1 [0/60000 (0%)]	Loss: 544.572815
Train Epoch: 1 [1280/60000 (2%)]	Loss: 217.949646
Train Epoch: 1 [2560/60000 (4%)]	Loss: 207.804291
Train Epoch: 1 [3840/60000 (6%)]	Loss: 204.570648
Train Epoch: 1 [5120/60000 (9%)]	Loss: 207.004913
Train Epoch: 1 [6400/60000 (11%)]	Loss: 200.568329
Train Epoch: 1 [7680/60000 (13%)]	Loss: 198.568146
Train Epoch: 1 [8960/60000 (15%)]	Loss: 195.804962
Train Epoch: 1 [10240/60000 (17%)]	Loss: 198.544037
Train Epoch: 1 [11520/60000 (19%)]	Loss: 194.491806
Train Epoch: 1 [12800/60000 (21%)]	Loss: 195.843216
Train Epoch: 1 [14080/60000 (23%)]	Loss: 186.054886
Train Epoch: 1 [15360/60000 (26%)]	Loss: 192.271210
Train Epoch: 1 [16640/60000 (28%)]	Loss: 188.747818
Train Epoch: 1 [17920/60000 (30%)]	Loss: 181.606552
Train Epoch: 1 [19200/60000 (32%)]	Loss: 173.035004
Train Epoch: 1 [20480/60000 (34%)]	Loss: 169.962662
Train Epoch: 1 [21760/60000 (36%)]	Loss: 178.848740
Train Epoch: 1 [23040/60000 (38%)]	Loss: 173.878723
Train Epoch: 1 [24320/60000 

Train Epoch: 4 [17920/60000 (30%)]	Loss: 105.239685
Train Epoch: 4 [19200/60000 (32%)]	Loss: 111.680817
Train Epoch: 4 [20480/60000 (34%)]	Loss: 106.347748
Train Epoch: 4 [21760/60000 (36%)]	Loss: 110.025711
Train Epoch: 4 [23040/60000 (38%)]	Loss: 111.148247
Train Epoch: 4 [24320/60000 (41%)]	Loss: 111.464821
Train Epoch: 4 [25600/60000 (43%)]	Loss: 107.021095
Train Epoch: 4 [26880/60000 (45%)]	Loss: 106.691879
Train Epoch: 4 [28160/60000 (47%)]	Loss: 107.595459
Train Epoch: 4 [29440/60000 (49%)]	Loss: 111.527725
Train Epoch: 4 [30720/60000 (51%)]	Loss: 111.588455
Train Epoch: 4 [32000/60000 (53%)]	Loss: 114.458031
Train Epoch: 4 [33280/60000 (55%)]	Loss: 110.340889
Train Epoch: 4 [34560/60000 (58%)]	Loss: 108.900345
Train Epoch: 4 [35840/60000 (60%)]	Loss: 113.034264
Train Epoch: 4 [37120/60000 (62%)]	Loss: 109.064178
Train Epoch: 4 [38400/60000 (64%)]	Loss: 110.082077
Train Epoch: 4 [39680/60000 (66%)]	Loss: 112.620773
Train Epoch: 4 [40960/60000 (68%)]	Loss: 111.578468
Train Epoch:

Train Epoch: 7 [35840/60000 (60%)]	Loss: 105.014236
Train Epoch: 7 [37120/60000 (62%)]	Loss: 104.870880
Train Epoch: 7 [38400/60000 (64%)]	Loss: 105.728806
Train Epoch: 7 [39680/60000 (66%)]	Loss: 104.188599
Train Epoch: 7 [40960/60000 (68%)]	Loss: 104.846916
Train Epoch: 7 [42240/60000 (70%)]	Loss: 103.260849
Train Epoch: 7 [43520/60000 (72%)]	Loss: 110.529999
Train Epoch: 7 [44800/60000 (75%)]	Loss: 104.750687
Train Epoch: 7 [46080/60000 (77%)]	Loss: 112.140343
Train Epoch: 7 [47360/60000 (79%)]	Loss: 105.811127
Train Epoch: 7 [48640/60000 (81%)]	Loss: 106.070694
Train Epoch: 7 [49920/60000 (83%)]	Loss: 107.849213
Train Epoch: 7 [51200/60000 (85%)]	Loss: 104.461441
Train Epoch: 7 [52480/60000 (87%)]	Loss: 103.509285
Train Epoch: 7 [53760/60000 (90%)]	Loss: 104.877289
Train Epoch: 7 [55040/60000 (92%)]	Loss: 105.373230
Train Epoch: 7 [56320/60000 (94%)]	Loss: 104.401436
Train Epoch: 7 [57600/60000 (96%)]	Loss: 105.848030
Train Epoch: 7 [58880/60000 (98%)]	Loss: 101.169281
====> Epoch:

Train Epoch: 10 [53760/60000 (90%)]	Loss: 105.911789
Train Epoch: 10 [55040/60000 (92%)]	Loss: 103.590469
Train Epoch: 10 [56320/60000 (94%)]	Loss: 105.020164
Train Epoch: 10 [57600/60000 (96%)]	Loss: 103.179832
Train Epoch: 10 [58880/60000 (98%)]	Loss: 105.625259
====> Epoch: 10 Average loss: 102.6906
====> Test set loss: 102.6855


In [9]:
num_gen = 16
with torch.no_grad():
    z = torch.randn(num_gen, ZDIMS).to(device)
    sample = model.decode(z).cpu()
    save_image(sample.view(num_gen, 1, 28, 28),
               'results/sample_' + str(epoch) + '.png')